In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/iot-dataset/IoT Network Intrusion Dataset.csv


In [2]:
!pip install optuna

In [3]:
!pip install catboost

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import warnings
import optuna
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import BernoulliNB
from lightgbm import LGBMClassifier
from sklearn.feature_selection import RFE
import itertools
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from tabulate import tabulate
import os
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras import regularizers


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [5]:
Maindataset = pd.read_csv("../input/iot-dataset/IoT Network Intrusion Dataset.csv")
Maindataset

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,192.168.0.13-192.168.0.16-10000-10101-17,192.168.0.13,10000,192.168.0.16,10101,17,25/07/2019 03:25:53 AM,75,1,1,...,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,Anomaly,Mirai,Mirai-Ackflooding
1,192.168.0.13-222.160.179.132-554-2179-6,222.160.179.132,2179,192.168.0.13,554,6,26/05/2019 10:11:06 PM,5310,1,2,...,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,Anomaly,DoS,DoS-Synflooding
2,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,11/07/2019 01:24:48 AM,141,0,3,...,0.0,0.0,0.0,70.5,0.707107,71.0,70.0,Anomaly,Scan,Scan Port OS
3,192.168.0.13-192.168.0.16-9020-52964-6,192.168.0.16,52964,192.168.0.13,9020,6,04/09/2019 03:58:17 AM,151,0,2,...,0.0,0.0,0.0,151.0,0.000000,151.0,151.0,Anomaly,Mirai,Mirai-Hostbruteforceg
4,192.168.0.1-239.255.255.250-36763-1900-17,192.168.0.1,36763,239.255.255.250,1900,17,10/09/2019 01:41:18 AM,153,2,1,...,0.0,0.0,0.0,76.5,0.707107,77.0,76.0,Anomaly,Mirai,Mirai-Hostbruteforceg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625778,192.168.0.24-210.89.164.90-56112-8043-17,192.168.0.24,56112,210.89.164.90,8043,17,25/07/2019 03:25:00 AM,277,1,1,...,0.0,0.0,0.0,277.0,0.000000,277.0,277.0,Anomaly,Mirai,Mirai-UDP Flooding
625779,192.168.0.13-222.131.171.244-554-4570-6,222.131.171.244,4570,192.168.0.13,554,6,26/05/2019 10:06:51 PM,1658,0,2,...,0.0,0.0,0.0,1658.0,0.000000,1658.0,1658.0,Anomaly,DoS,DoS-Synflooding
625780,192.168.0.13-192.168.0.16-9020-52739-6,192.168.0.16,52739,192.168.0.13,9020,6,11/07/2019 01:29:09 AM,77,1,1,...,0.0,0.0,0.0,77.0,0.000000,77.0,77.0,Anomaly,Scan,Scan Port OS
625781,192.168.0.13-192.168.0.16-9020-49784-6,192.168.0.13,9020,192.168.0.16,49784,6,20/05/2019 05:00:29 AM,240,2,1,...,0.0,0.0,0.0,120.0,7.071068,125.0,115.0,Normal,Normal,Normal


In [6]:

Maindataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625783 entries, 0 to 625782
Data columns (total 86 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow_ID            625783 non-null  object 
 1   Src_IP             625783 non-null  object 
 2   Src_Port           625783 non-null  int64  
 3   Dst_IP             625783 non-null  object 
 4   Dst_Port           625783 non-null  int64  
 5   Protocol           625783 non-null  int64  
 6   Timestamp          625783 non-null  object 
 7   Flow_Duration      625783 non-null  int64  
 8   Tot_Fwd_Pkts       625783 non-null  int64  
 9   Tot_Bwd_Pkts       625783 non-null  int64  
 10  TotLen_Fwd_Pkts    625783 non-null  float64
 11  TotLen_Bwd_Pkts    625783 non-null  float64
 12  Fwd_Pkt_Len_Max    625783 non-null  float64
 13  Fwd_Pkt_Len_Min    625783 non-null  float64
 14  Fwd_Pkt_Len_Mean   625783 non-null  float64
 15  Fwd_Pkt_Len_Std    625783 non-null  float64
 16  Bw

In [7]:
# Dropping columns that are not needed
Maindataset.drop(["Timestamp", "Src_IP", "Flow_ID", "Dst_IP"], axis = 1, inplace=True)


In [8]:
Maindataset

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,10000,10101,17,75,1,1,982.0,1430.0,982.0,982.0,...,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,Anomaly,Mirai,Mirai-Ackflooding
1,2179,554,6,5310,1,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,Anomaly,DoS,DoS-Synflooding
2,52727,9020,6,141,0,3,0.0,2806.0,0.0,0.0,...,0.0,0.0,0.0,70.5,0.707107,71.0,70.0,Anomaly,Scan,Scan Port OS
3,52964,9020,6,151,0,2,0.0,2776.0,0.0,0.0,...,0.0,0.0,0.0,151.0,0.000000,151.0,151.0,Anomaly,Mirai,Mirai-Hostbruteforceg
4,36763,1900,17,153,2,1,886.0,420.0,452.0,434.0,...,0.0,0.0,0.0,76.5,0.707107,77.0,76.0,Anomaly,Mirai,Mirai-Hostbruteforceg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625778,56112,8043,17,277,1,1,18.0,18.0,18.0,18.0,...,0.0,0.0,0.0,277.0,0.000000,277.0,277.0,Anomaly,Mirai,Mirai-UDP Flooding
625779,4570,554,6,1658,0,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1658.0,0.000000,1658.0,1658.0,Anomaly,DoS,DoS-Synflooding
625780,52739,9020,6,77,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,77.0,0.000000,77.0,77.0,Anomaly,Scan,Scan Port OS
625781,9020,49784,6,240,2,1,2776.0,1388.0,1388.0,1388.0,...,0.0,0.0,0.0,120.0,7.071068,125.0,115.0,Normal,Normal,Normal


In [9]:
# EDA


def initial_EDA(df):
  if isinstance(df, pd.DataFrame):

    total_na = df.isna().sum().sum()

    print("Dimension :%d rows, %d columns" % (df.shape[0], df.shape[1]))
    print("Total NA Values : %d " % (total_na))
    print("%38s %10s   %10s %10s" %
              ("Column Name", "Data Type", "#Distinct", "NA Values"))

    col_nam = df.columns
    dtype = df.dtypes
    uniq = df.nunique()
    na_val = df.isna().sum()

    for i in range(len(df.columns)):
      print("%38s %10s  %10s %10s" %
                  (col_nam[i], dtype[i], uniq[i], na_val[i]))

    else:

      print("Expact a DataFrame but got a %15s" % (type(df)))



In [10]:

initial_EDA(Maindataset)


Dimension :625783 rows, 82 columns
Total NA Values : 0 
                           Column Name  Data Type    #Distinct  NA Values
                              Src_Port      int64       11205          0
                              Dst_Port      int64        1034          0
                              Protocol      int64           3          0
                         Flow_Duration      int64       12670          0
                          Tot_Fwd_Pkts      int64         116          0
                          Tot_Bwd_Pkts      int64          42          0
                       TotLen_Fwd_Pkts    float64        2479          0
                       TotLen_Bwd_Pkts    float64        2612          0
                       Fwd_Pkt_Len_Max    float64        1156          0
                       Fwd_Pkt_Len_Min    float64        1165          0
                      Fwd_Pkt_Len_Mean    float64        2174          0
                       Fwd_Pkt_Len_Std    float64        1614      

In [11]:
# removing the missing values
Missingdata = pd.DataFrame({'total_missing': Maindataset.isnull().sum()})
Missingdata


,total_missing
Src_Port,0
Dst_Port,0
Protocol,0
Flow_Duration,0
Tot_Fwd_Pkts,0
...,...
Idle_Max,0
Idle_Min,0
Label,0
Cat,0


In [12]:
Maindataset = Maindataset.dropna()


In [13]:
# label encoding
categorical = [col for col in Maindataset.columns if Maindataset[col].dtypes == 'object']

categorical

['Label', 'Cat', 'Sub_Cat']

In [14]:

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Maindataset["Label"] = encoder.fit_transform(
    Maindataset["Label"])

Maindataset["Cat"] = encoder.fit_transform(
    Maindataset["Cat"])

Maindataset["Sub_Cat"] = encoder.fit_transform(
    Maindataset["Sub_Cat"])



In [15]:
Maindataset

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,10000,10101,17,75,1,1,982.0,1430.0,982.0,982.0,...,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,0,2,2
1,2179,554,6,5310,1,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,0,0,0
2,52727,9020,6,141,0,3,0.0,2806.0,0.0,0.0,...,0.0,0.0,0.0,70.5,0.707107,71.0,70.0,0,4,8
3,52964,9020,6,151,0,2,0.0,2776.0,0.0,0.0,...,0.0,0.0,0.0,151.0,0.000000,151.0,151.0,0,2,4
4,36763,1900,17,153,2,1,886.0,420.0,452.0,434.0,...,0.0,0.0,0.0,76.5,0.707107,77.0,76.0,0,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625778,56112,8043,17,277,1,1,18.0,18.0,18.0,18.0,...,0.0,0.0,0.0,277.0,0.000000,277.0,277.0,0,2,5
625779,4570,554,6,1658,0,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1658.0,0.000000,1658.0,1658.0,0,0,0
625780,52739,9020,6,77,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,77.0,0.000000,77.0,77.0,0,4,8
625781,9020,49784,6,240,2,1,2776.0,1388.0,1388.0,1388.0,...,0.0,0.0,0.0,120.0,7.071068,125.0,115.0,1,3,6


In [16]:
Maindataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625783 entries, 0 to 625782
Data columns (total 82 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Src_Port           625783 non-null  int64  
 1   Dst_Port           625783 non-null  int64  
 2   Protocol           625783 non-null  int64  
 3   Flow_Duration      625783 non-null  int64  
 4   Tot_Fwd_Pkts       625783 non-null  int64  
 5   Tot_Bwd_Pkts       625783 non-null  int64  
 6   TotLen_Fwd_Pkts    625783 non-null  float64
 7   TotLen_Bwd_Pkts    625783 non-null  float64
 8   Fwd_Pkt_Len_Max    625783 non-null  float64
 9   Fwd_Pkt_Len_Min    625783 non-null  float64
 10  Fwd_Pkt_Len_Mean   625783 non-null  float64
 11  Fwd_Pkt_Len_Std    625783 non-null  float64
 12  Bwd_Pkt_Len_Max    625783 non-null  float64
 13  Bwd_Pkt_Len_Min    625783 non-null  float64
 14  Bwd_Pkt_Len_Mean   625783 non-null  float64
 15  Bwd_Pkt_Len_Std    625783 non-null  float64
 16  Fl

In [17]:

# double checking the missing values
Maindataset_1 = pd.DataFrame({'total_missing': Maindataset.isnull().sum()})
Maindataset_1


,total_missing
Src_Port,0
Dst_Port,0
Protocol,0
Flow_Duration,0
Tot_Fwd_Pkts,0
...,...
Idle_Max,0
Idle_Min,0
Label,0
Cat,0


In [18]:
type(Maindataset_1)

pandas.core.frame.DataFrame

In [19]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)


In [20]:
cleanedDF = pd.DataFrame()

In [21]:
cleanedDF = clean_dataset(Maindataset)

In [22]:
cleanedDF.head()

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,10000.0,10101.0,17.0,75.0,1.0,1.0,982.0,1430.0,982.0,982.0,...,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,0.0,2.0,2.0
1,2179.0,554.0,6.0,5310.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,0.0,0.0,0.0
2,52727.0,9020.0,6.0,141.0,0.0,3.0,0.0,2806.0,0.0,0.0,...,0.0,0.0,0.0,70.5,0.707107,71.0,70.0,0.0,4.0,8.0
3,52964.0,9020.0,6.0,151.0,0.0,2.0,0.0,2776.0,0.0,0.0,...,0.0,0.0,0.0,151.0,0.000000,151.0,151.0,0.0,2.0,4.0
4,36763.0,1900.0,17.0,153.0,2.0,1.0,886.0,420.0,452.0,434.0,...,0.0,0.0,0.0,76.5,0.707107,77.0,76.0,0.0,2.0,4.0


In [23]:
X = cleanedDF.drop(['Label'], axis=1)
y = cleanedDF['Label']

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


# All the Feature Selections Methods 

In [25]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif

# Load your IoT dataset into a pandas DataFrame
# Assuming your dataset is stored in a variable called 'iot_data'
# Replace 'target_column' with the name of the target variable column in your dataset
# Replace the column names with the actual feature names in your dataset

# Select the top 5 features based on the ANOVA F-test
selector = SelectKBest(score_func=f_classif, k=5)
X_new = selector.fit_transform(X, y)

# Get the selected feature names
selected_features = X.columns[selector.get_support(indices=True)].tolist()

# Print the selected feature names
print("Selected features:", selected_features)


Selected features: ['Dst_Port', 'Fwd_Pkt_Len_Std', 'ACK_Flag_Cnt', 'Cat', 'Sub_Cat']


Feature selection is an important step in working with IoT datasets as it can help reduce the dimensionality of the data and improve the performance of machine learning models. Here are some common techniques for feature selection in IoT datasets:

1. Univariate feature selection: This method selects the best features based on univariate statistical tests such as chi-squared test, ANOVA F-test, or mutual information. Scikit-learn library in Python provides functions like SelectKBest and SelectPercentile for univariate feature selection.

2. Recursive feature elimination (RFE): RFE is a wrapper method that recursively removes the least important features and builds a model until the desired number of features is reached. This is available in scikit-learn's RFE class.

3. Lasso regularization: Lasso regularization (L1 regularization) can be used to penalize the absolute size of the regression coefficients, effectively performing feature selection by shrinking some coefficients to zero. This technique is commonly used in linear models.

4. Tree-based feature selection: Decision tree-based algorithms such as Random Forest or Gradient Boosting Machines can be used to rank the importance of features. You can then select the most important features based on their importance scores.

5. Principal Component Analysis (PCA): PCA is a dimensionality reduction technique that can also be used for feature selection by selecting the most important principal components that capture the most variance in the data.

When working with IoT datasets, it's important to consider the specific characteristics of the data and the goals of your analysis when selecting features. It's often a good idea to experiment with different feature selection methods and evaluate their impact on the performance of your machine learning models.

## The Start of AutoEncoder Models

In [26]:
x_scaled = MinMaxScaler().fit_transform(X_new)
x_normal_scaled = x_scaled[y==0]
x_fraud_scaled = x_scaled[y==1]


In [27]:
from sklearn.model_selection import train_test_split

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_new, y, test_size=0.3, random_state=0)

In [28]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

In [29]:
# Define the input layer
input_layer = Input(shape=(X_train_1.shape[1],))

# Define the encoder layer
encoded_layer = Dense(3, activation='relu')(input_layer)

# Define the decoder layer
decoded_layer = Dense(X_train_1.shape[1], activation='sigmoid')(encoded_layer)

# Define the autoencoder model
autoencoder = Model(input_layer, decoded_layer)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
autoencoder.fit(X_train_1, X_train_1, epochs=5, batch_size=32)

# Use the encoder layer to transform the input data into a lower-dimensional representation
encoder = Model(input_layer, encoded_layer)

X_encoded = encoder.predict(X_test_1)

Epoch 1/5
13681/13681 [==============================] - 36s 2ms/step - loss: 115511688.0000
Epoch 2/5
13681/13681 [==============================] - 33s 2ms/step - loss: 115512120.0000
Epoch 3/5
13681/13681 [==============================] - 33s 2ms/step - loss: 115511904.0000
Epoch 4/5
13681/13681 [==============================] - 33s 2ms/step - loss: 115511632.0000
Epoch 5/5
5864/5864 [==============================] - 6s 1ms/step


In [30]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.spatial import distance
from sklearn.metrics import r2_score

In [31]:
print(X_encoded.shape)
print(X_test_1.shape)

(187625, 3)
(187625, 5)


In [32]:
y_pred_reshaped = X_test_1[:, :3]
print(y_pred_reshaped.shape)

(187625, 3)


In [33]:
mse = mean_squared_error(y_pred_reshaped, X_encoded)
print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 448845194.4736574


In [34]:
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 21185.9669232645


In [35]:
mae = mean_absolute_error(y_pred_reshaped, X_encoded)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 13236.083792611926


In [36]:
cosine_similarities = [1 - distance.cosine(y_pred_reshaped[i], X_encoded[i]) for i in range(len(y_pred_reshaped))]
average_cosine_similarity = np.mean(cosine_similarities)
print("Average Cosine Similarity:", average_cosine_similarity)

Average Cosine Similarity: 0.012376641688448684


In [37]:
# Calculate the R-squared (coefficient of determination)
r_squared = r2_score(y_pred_reshaped, X_encoded)
print("R-squared (coefficient of determination):", r_squared)

R-squared (coefficient of determination): -73937753.2912723


# Stacked autoencoder 

In [38]:


# Define the input layer
input_layer = Input(shape=(X_train_1.shape[1],))

# Define the encoder layers with L2 regularization
encoded_layer1 = Dense(16, activation='relu', activity_regularizer=regularizers.l2(0.01))(input_layer)
encoded_layer2 = Dense(8, activation='relu', activity_regularizer=regularizers.l2(0.01))(encoded_layer1)
encoded_layer3 = Dense(4, activation='relu', activity_regularizer=regularizers.l2(0.01))(encoded_layer2)
encoded_layer4 = Dense(2, activation='relu', activity_regularizer=regularizers.l2(0.01))(encoded_layer3)
encoded_layer5 = Dense(1, activation='relu', activity_regularizer=regularizers.l2(0.01))(encoded_layer4)

# Define the decoder layers with L2 regularization
decoded_layer1 = Dense(2, activation='relu', activity_regularizer=regularizers.l2(0.01))(encoded_layer5)
decoded_layer2 = Dense(4, activation='relu', activity_regularizer=regularizers.l2(0.01))(decoded_layer1)
decoded_layer3 = Dense(8, activation='relu', activity_regularizer=regularizers.l2(0.01))(decoded_layer2)
decoded_layer4 = Dense(16, activation='relu', activity_regularizer=regularizers.l2(0.01))(decoded_layer3)
decoded_layer5 = Dense(X_train_1.shape[1], activation='sigmoid', activity_regularizer=regularizers.l2(0.01))(decoded_layer4)

# Define the autoencoder model
autoencoder = Model(input_layer, decoded_layer5)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
autoencoder.fit(X_train_1, X_train_1, epochs=5, batch_size=32)

# Use the encoder layers to transform the input data into a lower-dimensional representation
encoder = Model(input_layer, encoded_layer5)

X_encoded_1 = encoder.predict(X_test_1)

# Print the shape of the encoded data
print("Encoded data shape:", X_encoded_1.shape)

Epoch 1/5
13681/13681 [==============================] - 76s 5ms/step - loss: 115559472.0000
Epoch 2/5
13681/13681 [==============================] - 72s 5ms/step - loss: 115511584.0000
Epoch 3/5
13681/13681 [==============================] - 72s 5ms/step - loss: 115511640.0000
Epoch 4/5
13681/13681 [==============================] - 71s 5ms/step - loss: 115511720.0000
Epoch 5/5
5864/5864 [==============================] - 8s 1ms/step
Encoded data shape: (187625, 1)


In [39]:
x_reshaped = X_test_1[:, :1]
print(x_reshaped.shape)

(187625, 1)


In [40]:
mse_1 = mean_squared_error(x_reshaped, X_encoded_1)
print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 448845194.4736574


In [41]:
rmse = np.sqrt(mse_1)
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 23982.145027407318


In [42]:
mae = mean_absolute_error(x_reshaped, X_encoded_1)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 16363.811437708195


In [43]:
cosine_similarities = [1 - distance.cosine(x_reshaped[i], X_encoded_1[i]) for i in range(len(y_pred_reshaped))]
average_cosine_similarity = np.mean(cosine_similarities)
print("Average Cosine Similarity:", average_cosine_similarity)

Average Cosine Similarity: 1.0


In [44]:
# Calculate the R-squared (coefficient of determination)
r_squared = r2_score(x_reshaped, X_encoded_1)
print("R-squared (coefficient of determination):", r_squared)

R-squared (coefficient of determination): -0.8711820764943756


## LSTM auto encoder 

In [45]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

# Load your IoT dataset into a pandas DataFrame
# Assuming your dataset is stored in a variable called 'iot_data'
# Replace 'target_column' with the name of the target variable column in your dataset
# Replace the column names with the actual feature names in your dataset
# X = iot_data.drop(['target_column'], axis=1)  # Features
# y = iot_data['target_column']  # Target variable

# Select the top 5 features based on the ANOVA F-test
# selector = SelectKBest(score_func=f_classif, k=5)
# X_new = selector.fit_transform(X, y)

# Define the input layer
input_layer = Input(shape=(X_train.shape[1],))

# Define the stacked encoder LSTM layers with dropout and L2 regularization
encoded = LSTM(16, activation='relu', kernel_regularizer=regularizers.l2(0.01), return_sequences=True)(input_layer)
encoded = Dropout(0.2)(encoded)
encoded = LSTM(8, activation='relu', kernel_regularizer=regularizers.l2(0.01), return_sequences=True)(encoded)
encoded = Dropout(0.2)(encoded)
encoded = LSTM(4, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)

# Define the stacked decoder LSTM layers with dropout and L2 regularization
decoded = RepeatVector(X_train.shape[1])(encoded)
decoded = LSTM(8, activation='relu', kernel_regularizer=regularizers.l2(0.01), return_sequences=True)(decoded)
decoded = Dropout(0.2)(decoded)
decoded = LSTM(16, activation='relu', kernel_regularizer=regularizers.l2(0.01), return_sequences=True)(decoded)
decoded = Dropout(0.2)(decoded)
decoded = LSTM(X_train.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01), return_sequences=True)(decoded)

# Define the autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
autoencoder.fit(X_train_1, X_train_1, epochs=50, batch_size=32)

# Use the encoder layer to transform the input data into a lower-dimensional representation
encoder = Model(input_layer, encoded)
X_encoded_3 = encoder.predict(X_test_1)

# Print the shape of the encoded data
print("Encoded data shape:", X_encoded_3.shape)

ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 81)

# Deep Autoencoding Guassian Mixture Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

# Load your IoT dataset into a pandas DataFrame
# Assuming your dataset is stored in a variable called 'iot_data'
# Replace 'target_column' with the name of the target variable column in your dataset
# Replace the column names with the actual feature names in your dataset
# X = iot_data.drop(['target_column'], axis=1)  # Features
# y = iot_data['target_column']  # Target variable

# Standardize the input data
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

# Define the input layer
input_layer = Input(shape=(X_train_1.shape[1],))

# Define the deep autoencoder architecture
encoded = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(input_layer)
encoded = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)
encoded = Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)
encoded = Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)

decoded = Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)
decoded = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(decoded)
decoded = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(decoded)
decoded = Dense(X_train_1.shape[1], activation='linear')(decoded)

# Define the autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
autoencoder.fit(X_train_1, X_train_1, epochs=50, batch_size=32)

# Use the encoder layer to transform the input data into a lower-dimensional representation
encoder = Model(input_layer, encoded)
X_encoded_4 = encoder.predict(X_test_1)

# Fit a Gaussian Mixture Model to the encoded data
gmm = GaussianMixture(n_components=3)  # Set the number of components as needed
gmm.fit(X_encoded_4)

# Predict the GMM component assignments for the encoded data
component_assignments = gmm.predict(X_encoded_4)

# Print the component assignments
print("Component Assignments:", component_assignments)

## Autoencoder with Deep Belief Classifier

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

# Load your IoT dataset into a pandas DataFrame
# Assuming your dataset is stored in a variable called 'iot_data'
# Replace 'target_column' with the name of the target variable column in your dataset
# Replace the column names with the actual feature names in your dataset
# X = iot_data.drop(['target_column'], axis=1)  # Features
# y = iot_data['target_column']  # Target variable

# Standardize the input data
# scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

# Define the input layer
input_layer = Input(shape=(X_train_1.shape[1],))

# Define the deep autoencoder architecture
encoded = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(input_layer)
encoded = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)
encoded = Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)
encoded = Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)

decoded = Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)
decoded = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(decoded)
decoded = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(decoded)
decoded = Dense(X_train_1.shape[1], activation='linear')(decoded)

# Define the autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
autoencoder.fit(X_train_1, X_train_1, epochs=50, batch_size=32)

# Use the encoder layer to transform the input data into a lower-dimensional representation
encoder = Model(input_layer, encoded)
X_encoded_5 = encoder.predict(X_test_1)

# Define the Deep Belief Network (DBN) classifier
rbm = BernoulliRBM(n_components=64, learning_rate=0.01, batch_size=10, n_iter=10, verbose=0, random_state=42)
logistic = LogisticRegression(solver='newton-cg', tol=1, multi_class='auto')
dbn_classifier = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])

# Train the DBN classifier on the encoded data
dbn_classifier.fit(X_encoded_5, X_encoded_5)

# Make predictions using the DBN classifier
predictions = dbn_classifier.predict(X_encoded_5)

# Print the predictions
print("DBN Classifier Predictions:", predictions)

# Adversarial autoencoder

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense, Lambda, concatenate, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

# Load your IoT dataset into a pandas DataFrame
# Assuming your dataset is stored in a variable called 'iot_data'
# Replace 'target_column' with the name of the target variable column in your dataset
# Replace the column names with the actual feature names in your dataset
X = iot_data.drop(['target_column'], axis=1)  # Features
y = iot_data['target_column']  # Target variable

# Standardize the input data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the input layer
input_layer = Input(shape=(X_scaled.shape[1],))

# Define the encoder architecture
encoded = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(input_layer)
encoded = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)
encoded = Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)
encoded = Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)

# Define the decoder architecture
decoded = Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)
decoded = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(decoded)
decoded = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(decoded)
decoded = Dense(X_scaled.shape[1], activation='linear')(decoded)

# Define the autoencoder model
autoencoder = Model(input_layer, decoded)

# Define the discriminator architecture
discriminator_input = Input(shape=(8,))
discriminator = Dense(16, activation='relu')(discriminator_input)
discriminator = Dense(1, activation='sigmoid')(discriminator)

# Define the adversarial autoencoder model
adversarial_autoencoder = Model(input_layer, [decoded, discriminator(encoded)])

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Compile the discriminator model
discriminator.compile(optimizer=Adam(0.0001), loss='binary_crossentropy')

# Freeze the autoencoder weights before training the discriminator
for layer in autoencoder.layers:
    layer.trainable = False

# Combine the autoencoder and discriminator models
adversarial_autoencoder.compile(optimizer='adam', loss=['mean_squared_error', 'binary_crossentropy'], loss_weights=[0.999, 0.001])

# Train the adversarial autoencoder
for epoch in range(50):
    # Train the discriminator
    discriminator_loss = []
    for i in range(100):
        # Generate a batch of real and fake data
        real_data = X_encoded[np.random.randint(0, X_encoded.shape[0], size=32)]
        fake_data = K.eval(encoded(K.constant(X_scaled[np.random.randint(0, X_scaled.shape[0], size=32)])))
        # Train the discriminator on the real and fake data
        discriminator_loss.append(discriminator.train_on_batch(real_data, np.ones((32, 1))))
        discriminator_loss.append(discriminator.train_on_batch(fake_data, np.zeros((32, 1)))))

    # Train the autoencoder with the discriminator frozen
    for layer in autoencoder.layers:
        layer.trainable = False
    autoencoder_loss = autoencoder.train_on_batch(X_scaled, X_scaled)

    # Train the adversarial autoencoder
    for layer in autoencoder.layers:
        layer.trainable = True
    adversarial_autoencoder_loss = adversarial_autoencoder.train_on_batch(X_scaled, [X_scaled, np.ones((X_scaled.shape[0], 1))])

    # Print the loss for each epoch
    print('Epoch %d: Autoencoder Loss = %f, Discriminator Loss = %f, Adversarial Loss = %f' % (epoch, autoencoder_loss, np.mean(discriminator_loss), adversarial_autoencoder_loss[0]))

# Sparse Autoencoder

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

# Define the input layer
input_layer = Input(shape=(input_dim,))

# Define the encoder architecture with sparsity constraint
encoded = Dense(64, activation='relu', activity_regularizer=regularizers.l1(0.01))(input_layer)
encoded = Dense(32, activation='relu', activity_regularizer=regularizers.l1(0.01))(encoded)
encoded = Dense(16, activation='relu', activity_regularizer=regularizers.l1(0.01))(encoded)
encoded = Dense(8, activation='relu', activity_regularizer=regularizers.l1(0.01))(encoded)

# Define the decoder architecture
decoded = Dense(16, activation='relu')(encoded)
decoded = Dense(32, activation='relu')(decoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(input_dim, activation='linear')(decoded)

# Define the autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Contractive autoencoder

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

def contractive_autoencoder(encoding_dim, lam=1e-4):
    # Define the input layer
    inputs = Input(shape=(X_train.shape[1],))
    
    # Encoder
    encoded = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l2(lam))(inputs)
    
    # Decoder
    decoded = Dense(X_train.shape[1], activation='sigmoid')(encoded)
    
    # Define the contractive loss function
    def contractive_loss(y_true, y_pred):
        mse = K.mean(K.square(y_true - y_pred), axis=1)
        
        # Compute the Jacobian of the encoder output with respect to the input
        encoder = Model(inputs, encoded)
        with tf.GradientTape() as tape:
            tape.watch(inputs)
            encoded_output = encoder(inputs)
        jacobian = tape.batch_jacobian(encoded_output, inputs)
        jacobian_norm = K.sqrt(K.sum(K.square(jacobian), axis=(1, 2)))
        
        # Compute the contractive loss
        return mse + lam * jacobian_norm
    
    # Create the autoencoder model
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer=Adam(), loss=contractive_loss)
    
    return autoencoder

# Instantiate the contractive autoencoder model
encoding_dim = 32  # Dimension of the encoded representation
ca_model = contractive_autoencoder(encoding_dim)

# Example synthetic data (as placeholder for X_train)
X_train = np.random.rand(1000, 20)  # 1000 samples, 20 features

# Train the contractive autoencoder
ca_model.fit(X_train, X_train, epochs=50, batch_size=32)

print("Training complete!")


## LSTM auto encoder 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

# Load your IoT dataset into a pandas DataFrame
# Assuming your dataset is stored in a variable called 'iot_data'
# Replace 'target_column' with the name of the target variable column in your dataset
# Replace the column names with the actual feature names in your dataset
X = iot_data.drop(['target_column'], axis=1)  # Features
y = iot_data['target_column']  # Target variable

# Select the top 5 features based on the ANOVA F-test
selector = SelectKBest(score_func=f_classif, k=5)
X_new = selector.fit_transform(X, y)

# Define the input layer
input_layer = Input(shape=(X_new.shape[1],))

# Define the encoder LSTM layers with dropout and L2 regularization
encoded = LSTM(16, activation='relu', kernel_regularizer=regularizers.l2(0.01), return_sequences=True)(input_layer)
encoded = Dropout(0.2)(encoded)
encoded = LSTM(8, activation='relu', kernel_regularizer=regularizers.l2(0.01), return_sequences=True)(encoded)
encoded = Dropout(0.2)(encoded)
encoded = LSTM(4, activation='relu', kernel_regularizer=regularizers.l2(0.01))(encoded)

# Repeat the encoded representation to the original sequence length
decoded = RepeatVector(X_new.shape[1])(encoded)

# Define the decoder LSTM layers with dropout and L2 regularization
decoded = LSTM(8, activation='relu', kernel_regularizer=regularizers.l2(0.01), return_sequences=True)(decoded)
decoded = Dropout(0.2)(decoded)
decoded = LSTM(16, activation='relu', kernel_regularizer=regularizers.l2(0.01), return_sequences=True)(decoded)
decoded = Dropout(0.2)(decoded)
decoded = LSTM(X_new.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01), return_sequences=True)(decoded)

# Define the autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
autoencoder.fit(X_new, X_new, epochs=50, batch_size=32)

# Use the encoder layer to transform the input data into a lower-dimensional representation
encoder = Model(input_layer, encoded)
X_encoded = encoder.predict(X_new)

# Print the shape of the encoded data
print("Encoded data shape:", X_encoded.shape)

In [ ]:

autoencoder = Model(input_layer, output_layer)

# the compile section
autoencoder.compile(optimizer = 'adam', loss='mse')


In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(autoencoder, 'autoencoder_no_compress.png', show_shapes=True)

In [ ]:
history = autoencoder.fit(x_normal_scaled, x_normal_scaled, batch_size=16, epochs=30, shuffle=True, validation_split = 0.20)

In [ ]:
# plotting the result
from matplotlib import pyplot

pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()
encoder = Model(input_layer, output_layer)
plot_model(encoder, 'encoder_no_compress.png', show_shapes=True)
encoder.save('encoder.h5')

# LSTM Auto Encoder

In [ ]:
# Creating a 3D dataset
TIME_STEPS = 30

def create_sequences(X_1, y_1, time_steps=TIME_STEPS):

  Xs, ys = [], []

  for i in range(len(X_1)-time_steps):

    Xs.append(X.iloc[i:(i+time_steps)].values)
    ys.append(y.iloc[i+time_steps])

  return np.array(Xs), np.array(ys)

X_train_1, y_train_1 = create_sequences(X_train, y_train)

X_test_1, y_test_1 = create_sequences(X_test, y_test)



In [ ]:
print(X_test_1.shape)
print(X_train_1.shape)

In [ ]:
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go

np.random.seed(1)
tf.random.set_seed(1)

from tensorflow.keras.models import Sequential


In [ ]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed, Dropout



model = Sequential()
model.add(LSTM(128, input_shape=(X_train_1.shape[1], X_train_1.shape[2])))
model.add(Dropout(rate=0.2))
model.add(RepeatVector(X_train_1.shape[1]))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(TimeDistributed(Dense(X_train_1.shape[2])))
model.compile(optimizer='adam', loss='mae')
model.summary()



In [ ]:
history = model.fit(X_train_1, y_train_1, epochs=2, batch_size=16, validation_split=0.1,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min')], shuffle=False)